In [1]:
import pandas as pd
import numpy as np
import os

In [3]:
cd models

F:\FINAL\Prediction\pc_ui2\FINAL\Prediction\models


In [4]:
# folders = [_ for _ in os.listdir('./') if 'final' in _ and 'norm' in _]
folders = [_ for _ in os.listdir('./') if 'mae' in _]

In [5]:
folders

['CNN_multioutput_lat_long_depth_magnitude_time_5s-6s_mae_B_4_3x1024_orto_par512-256',
 'EFFICIENTNETB5_multioutput_lat_long_depth_magnitude_time_5s-6s_mae_B_4_1x1024orto_paral512-256',
 'MOBILENETV3_multioutput_lat_long_depth_magnitude_time_5s-6s_mae_B_4_1x1024orto_paral512-256']

In [6]:
df_hasil = {}

for j,folder in enumerate(folders):
    files_folder = os.listdir('./'+folder)
    print(j,folder,':',len(files_folder))
    if len(files_folder)>=6:
        for file in files_folder:
            if 'SWA.csv' in file:
                df_hasil['swa_'+folder] = pd.read_csv('./'+folder+'/'+file)
            elif 'best.csv' in file:
                df_hasil['best_'+folder] = pd.read_csv('./'+folder+'/'+file)
    else:
        print(len(files_folder))


0 CNN_multioutput_lat_long_depth_magnitude_time_5s-6s_mae_B_4_3x1024_orto_par512-256 : 12
1 EFFICIENTNETB5_multioutput_lat_long_depth_magnitude_time_5s-6s_mae_B_4_1x1024orto_paral512-256 : 12
2 MOBILENETV3_multioutput_lat_long_depth_magnitude_time_5s-6s_mae_B_4_1x1024orto_paral512-256 : 12


In [8]:
### ADD FUNCTION MAPE
def mean_absolute_percentage_error(y_true, y_pred,multioutput):
    epsilon = np.finfo(np.float64).eps
    mape = np.abs(y_pred - y_true) / np.maximum(np.abs(y_true), epsilon)
    output_errors = np.average(mape,
                               weights=sample_weight, axis=0)
    if isinstance(multioutput, str):
        if multioutput == 'raw_values':
            return output_errors
        elif multioutput == 'uniform_average':
            # pass None as weights to np.average: uniform mean
            multioutput = None

    return np.average(output_errors)

In [9]:
from sklearn.metrics import r2_score as R2
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_squared_error as MSE
# from sklearn.metrics import mean_absolute_percentage_error as MAPE

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    e_ = epsilon = np.finfo(np.float64).eps
    y_true = np.where(y_true==0,y_true+e_,y_true)
    return np.mean(np.abs((y_true - y_pred) / y_true))

MAPE = mean_absolute_percentage_error

outnames = ['lat','long','depth','magnitude','time']

ds_scores = {'modelname':list(df_hasil.keys())}
ds_scores['r2_variance'] = []
for col in ['mean']+outnames:
    ds_scores['r2_'+col] = []
    ds_scores['rmse_'+col] = []
    ds_scores['mse_'+col] = []
    ds_scores['mae_'+col] = []
    ds_scores['mape_'+col] = []


for key in df_hasil.keys():
    col_names = [_ for _ in key.split('_') if _ in outnames]
    col_norm_names = ['norm_'+col for col in col_names]
    col_pred_names = ['pred_'+col for col in col_names]
    y0 = df_hasil[key][col_norm_names].values
    y1 = df_hasil[key][col_pred_names].values
    
    ds_scores['r2_variance'] += [R2(y0,y1,multioutput='variance_weighted')] 
    ds_scores['r2_mean'] += [R2(y0,y1,multioutput='uniform_average')]  
    ds_scores['rmse_mean'] += [MSE(y0,y1,multioutput='uniform_average',squared=False)]  
    ds_scores['mse_mean'] += [MSE(y0,y1,multioutput='uniform_average')]    
    ds_scores['mae_mean'] += [MAE(y0,y1,multioutput='uniform_average')]    

    singlesR2 = R2(y0,y1,multioutput='raw_values')
    singlesMSE = MSE(y0,y1,multioutput='raw_values')
    singlesMAE = MAE(y0,y1,multioutput='raw_values')

    for_mape = []
    for j,col_name in enumerate(col_names):
        col_norm_names = ['norm_'+col for col in [col_name]]
        col_pred_names = ['pred_'+col for col in [col_name]]
        y0 = df_hasil[key][col_norm_names].values
        y1 = df_hasil[key][col_pred_names].values
        for_mape += [MAPE(y0,y1)]
    singlesMAPE = np.array(for_mape)
    
    ds_scores['mape_mean'] += [np.mean(singlesMAPE)]

    jj = 0
    for j,col in enumerate(outnames):
        if col in col_names:  
            try:
                ds_scores['r2_'+col] += [singlesR2[j]]
                ds_scores['rmse_'+col] += [np.sqrt(singlesMSE[j])]
                ds_scores['mse_'+col] += [singlesMSE[j]]
                ds_scores['mae_'+col] += [singlesMAE[j]]
                ds_scores['mape_'+col] += [singlesMAPE[j]]
                jj += 1
            except:
                ds_scores['r2_'+col] += [singlesR2[jj]]
                ds_scores['rmse_'+col] += [np.sqrt(singlesMSE[jj])]
                ds_scores['mse_'+col] += [singlesMSE[jj]]
                ds_scores['mae_'+col] += [singlesMAE[jj]]
                ds_scores['mape_'+col] += [singlesMAPE[jj]]
        else:
            ds_scores['r2_'+col] += [np.nan]
            ds_scores['rmse_'+col] += [np.nan]
            ds_scores['mse_'+col] += [np.nan]
            ds_scores['mae_'+col] += [np.nan]
            ds_scores['mape_'+col] += [np.nan]

In [10]:
## create dataframe rekap
df_rekap = pd.DataFrame(ds_scores)

In [13]:
df_rekap.to_csv('rekap_test.csv',index=False)
df_rekap.to_excel('rekap_test.xlsx')

In [12]:
df_rekap

,modelname,r2_variance,r2_mean,rmse_mean,mse_mean,mae_mean,mape_mean,r2_lat,rmse_lat,mse_lat,...,r2_magnitude,rmse_magnitude,mse_magnitude,mae_magnitude,mape_magnitude,r2_time,rmse_time,mse_time,mae_time,mape_time
0,best_CNN_multioutput_lat_long_depth_magnitude_...,0.791670,0.565257,0.072143,0.005205,0.049707,3.727599e+12,0.737311,0.088929,0.007908,...,0.720768,0.083807,0.007024,0.063163,1.863799e+13,0.811684,0.056997,0.003249,0.041961,0.251120
1,swa_CNN_multioutput_lat_long_depth_magnitude_t...,0.785766,0.549646,0.073158,0.005352,0.051512,3.357237e+12,0.714840,0.092655,0.008585,...,0.734791,0.081675,0.006671,0.061656,1.678619e+13,0.800930,0.058601,0.003434,0.044980,0.269381
2,best_EFFICIENTNETB5_multioutput_lat_long_depth...,0.864543,0.698658,0.058173,0.003384,0.041447,2.975148e+12,0.903573,0.053879,0.002903,...,0.781718,0.074098,0.005490,0.055857,1.487574e+13,0.818655,0.055932,0.003128,0.040599,0.226495
3,swa_EFFICIENTNETB5_multioutput_lat_long_depth_...,0.847899,0.675713,0.061643,0.003800,0.042688,3.550264e+12,0.887895,0.058095,0.003375,...,0.729568,0.082476,0.006802,0.063991,1.775132e+13,0.810035,0.057246,0.003277,0.040912,0.246689
4,best_MOBILENETV3_multioutput_lat_long_depth_ma...,0.827724,0.637788,0.065604,0.004304,0.046550,3.836747e+12,0.843138,0.068720,0.004722,...,0.745485,0.080012,0.006402,0.061943,1.918373e+13,0.781330,0.061418,0.003772,0.045732,0.260229
5,swa_MOBILENETV3_multioutput_lat_long_depth_mag...,0.808418,0.583165,0.069183,0.004786,0.048798,3.914384e+12,0.819185,0.073780,0.005444,...,0.705409,0.086081,0.007410,0.067585,1.957192e+13,0.778956,0.061751,0.003813,0.044989,0.261758
